# Data augmentation

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import multiprocessing
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from main import MNIST_dataset, MNIST_trainer

In [3]:
print("Torch version: ", torch.__version__)

####################################################################
# Set Device
####################################################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", device)

Torch version:  2.5.1+cu124
Device:  cuda


In [4]:
####################################################################
# DataLoader Class
####################################################################
da = transforms.Compose(
    [
        transforms.RandomRotation((15)),
        transforms.ElasticTransform(alpha=110.0),
        transforms.ToTensor(),
    ]
)
train_dataset = MNIST_dataset(partition="train", da_transform=da)
test_dataset = MNIST_dataset(partition="test")

batch_size = 100
num_workers = multiprocessing.cpu_count() - 1
print("Num workers", num_workers)

train_dataloader = DataLoader(
    train_dataset, batch_size, shuffle=True, num_workers=num_workers
)
test_dataloader = DataLoader(
    test_dataset, batch_size, shuffle=False, num_workers=num_workers
)


Loading MNIST  train  Dataset...
	Total Len.:  60000 
 --------------------------------------------------

Loading MNIST  test  Dataset...
	Total Len.:  10000 
 --------------------------------------------------
Num workers 11


In [ ]:
####################################################################
# Neural Network Class
####################################################################


# Creating our Neural Network - Fully Connected
class Net(nn.Module):
    def __init__(
        self,
        sizes=[[784, 1024], [1024, 1024], [1024, 1024], [1024, 10]],
        criterion=None,
    ):
        super(Net, self).__init__()

        self.layers = nn.ModuleList()

        for i in range(len(sizes) - 1):
            dims = sizes[i]
            self.layers.append(nn.Linear(dims[0], dims[1]))
            self.layers.append(nn.BatchNorm1d(dims[1]))
            self.layers.append(nn.ReLU())

        dims = sizes[-1]
        self.classifier = nn.Linear(dims[0], dims[1])

        self.criterion = criterion

    def forward(self, x, y=None):
        for layer in self.layers:
            x = layer(x)
        x = self.classifier(x)

        if y != None:
            loss = self.criterion(x, y)
            return loss, x
        return x


####################################################################
# Training settings
####################################################################

# Training hyperparameters
criterion = nn.CrossEntropyLoss()
# Instantiating the network and printing its architecture
num_classes = 10
net = Net(
    sizes=[[784, 1024], [1024, 1024], [1024, 1024], [1024, num_classes]],
    criterion=criterion,
)
print(net)


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print("Params: ", count_parameters(net))
optimizer = optim.SGD(net.parameters(), lr=0.01, weight_decay=1e-6, momentum=0.9)
epochs = 50

scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, "min", patience=5, threshold=0.001, cooldown=1, verbose=True
)

trainer = MNIST_trainer(
    net,
    train_dataloader,
    test_dataloader,
    optimizer,
    criterion,
    epochs,
    device,
    scheduler=scheduler,
    model_path="models/da2.pt",
)

Net(
  (layers): ModuleList(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=1024, out_features=1024, bias=True)
    (4): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=1024, out_features=1024, bias=True)
    (7): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
  )
  (classifier): Linear(in_features=1024, out_features=10, bias=True)
  (criterion): CrossEntropyLoss()
)
Params:  2919434


/home/diego/.cache/pypoetry/virtualenvs/mlps-yB44cTBw-py3.10/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [6]:
####################################################################
# Training
####################################################################

trainer.train()


---- Start Training ----


Test 0: 100%|██████████| 100/100 [00:00<00:00, 107.18batch/s]


	LR:  0.01
[Epoch 1] Train Loss: 0.008098 - Test Loss: 0.001062 - Train Accuracy: 73.34% - Test Accuracy: 96.58%


Test 1: 100%|██████████| 100/100 [00:00<00:00, 111.84batch/s]

	LR:  0.01
[Epoch 2] Train Loss: 0.005373 - Test Loss: 0.000916 - Train Accuracy: 82.58% - Test Accuracy: 97.05%


...

	LR:  1.0000000000000004e-08
[Epoch 98] Train Loss: 0.001470 - Test Loss: 0.000209 - Train Accuracy: 95.28% - Test Accuracy: 99.26%



Test 98: 100%|██████████| 100/100 [00:00<00:00, 121.21batch/s]

	LR:  1.0000000000000004e-08
[Epoch 99] Train Loss: 0.001467 - Test Loss: 0.000207 - Train Accuracy: 95.37% - Test Accuracy: 99.23%



Test 99: 100%|██████████| 100/100 [00:00<00:00, 110.56batch/s]

	LR:  1.0000000000000004e-08
[Epoch 100] Train Loss: 0.001467 - Test Loss: 0.000213 - Train Accuracy: 95.31% - Test Accuracy: 99.25%

BEST TEST ACCURACY:  99.29  in epoch  45


In [7]:
####################################################################
# Load best weights
####################################################################

trainer.get_model()

/home/diego/code/RNA/mnist_tests/main.py:161: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.net.load_state_dict(torch.load(self.model_path))
Test 99: 100%|██████████| 1

Final best acc:  99.29
